# Preparing Mistral NeMo for Amazon Bedrock Custom Model Import (CMI)

This notebook demonstrates how to prepare and import Mistral NeMo into Amazon Bedrock using Custom Model Import (CMI).

## Prerequisites

1. **HuggingFace Access**
   - Active HuggingFace account
   - Valid access token
   - CLI authentication with HuggingFace (required for file transfers)

2. **File Transfer Method**
   - This notebook uses HF Transfer for efficient direct transfers from HuggingFace
   - Alternative: Manual download and S3 upload

3. **Model Configuration Requirements**
   - Must set `max_position_embeddings` to 8192 or less to comply with Bedrock limits
   - Defines the maximum sequence length

4. **File Format Requirements**
   - All model files must be in HuggingFace format
   - Required files include:
     - Model weights (.safetensors)
     - Configuration files (config.json, generation_config.json)
     - Tokenizer files (tokenizer.json, tokenizer_config.json)
     - Supporting files (vocab.json, merges.txt, special_tokens_map.json)
5. **EBS Volume**
   - If the notebook is run from Sagemaker Notebook instance, increase the EBS volume to 100gb

## Important Note on Model Precision
Bedrock CMI has specific requirements for model precision:
- Supported: FP32, FP16, and BF16 precision
- Not supported: Quantized models (including 4-bit quantization)
- Note: FP32 models will be automatically converted to BF16 precision internally by Bedrock

In [ ]:
!pip install -q transformers hf_transfer huggingface_hub

## Create IAM role and policy for CMI

In [ ]:
import boto3
import json

iam = boto3.client("iam")

iam.create_role(
    RoleName="MyImportModelRole",
    AssumeRolePolicyDocument=json.dumps({
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "bedrock.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ] 
    })
)

iam.create_policy(
    PolicyName="S3BucketPolicy",
    PolicyDocument=json.dumps({
    "Version": "2012-10-17",
        "Statement": [
            {
            "Effect": "Allow",
            "Action": [
                "s3:PutObject",
                "s3:GetObject",
                "s3:ListBucket",
                "s3:DeleteObject"
                ],
            "Resource": [
            "arn:aws:s3:::{YOUR_S3_BUCKET}",
            "arn:aws:s3:::{YOUR_S3_BUCKET}/*"
            ]
            }
        ]
}
)
     )

## Attach Policy to the Role

In [ ]:
iam.attach_role_policy(
    RoleName="MyImportModelRole",
    PolicyArn="YOUR_POLICY_ARN" #the previous cell will display the ARN
)

## Download and Upload Hugging Face Model Files to S3 using HF Transfer

In [ ]:
import os
import boto3
from huggingface_hub import hf_hub_download, login, snapshot_download
import json
import boto3

HF_TOKEN = "YOUR_HF_TOKEN"
login(token = HF_TOKEN)

# Enable the faster transfers
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

# Repository and files
repo_id = "mistralai/Mistral-Nemo-Instruct-2407"

# S3 configuration
bucket_name = "YOUR_BUCKET_NAME"
prefix = "YOUR_PREFIX"

# Download location
temp_dir = "./temp_model_files"
os.makedirs(temp_dir, exist_ok=True)

# Initialize S3 client
s3_client = boto3.client('s3')

snapshot_path = snapshot_download(
    repo_id="mistralai/Mistral-Nemo-Instruct-2407",
    local_dir=temp_dir,
    ignore_patterns=["consolidated.safetensors"]
)


for root, dirs, files in os.walk(snapshot_path):
    for file in files:
        local_file_path = os.path.join(root, file)
        # Compute S3 key to preserve relative path
        rel_path = os.path.relpath(local_file_path, snapshot_path)
        s3_key = os.path.join(prefix, rel_path)
        print(f"Uploading {local_file_path} to s3://{bucket_name}/{s3_key}...")
        try:
            s3_client.upload_file(
                Filename=local_file_path,
                Bucket=bucket_name,
                Key=s3_key
            )
            print(f"Successfully uploaded {rel_path}")
        except Exception as e:
            print(f"Error uploading {rel_path}: {e}")



## Update max_position_embeddings

In [ ]:
# Initialize S3 client
s3 = boto3.client('s3')

# S3 details
bucket_name = 'YOUR_BUCKET_NAME'
config_key = '{PREFIX}/config.json'  # full path to config.json

# Download the current config
response = s3.get_object(Bucket=bucket_name, Key=config_key)
config = json.loads(response['Body'].read().decode('utf-8'))

# Modify the config
config['max_position_embeddings'] = 8192  # Using Bedrock's recommended value

# Upload modified config back to S3
s3.put_object(
    Bucket=bucket_name,
    Key=config_key,
    Body=json.dumps(config, indent=2),
    ContentType='application/json'
)

print("Config updated successfully")

## Start CMI Import Process

In [ ]:
import boto3
import time
import json

# Initialize Bedrock client
bedrock = boto3.client('bedrock')

def start_model_import(
    bucket_name="YOUR_BUCKET_NAME", 
    prefix="PREFIX",
    model_name="mistral-nemo-test",
    role_arn="YOUR_ROLE_ARN" 
):
    
    # Construct S3 URI
    s3_uri = f"s3://{bucket_name}/{prefix}"
    
    try:
        response = bedrock.create_model_import_job(
            importedModelName=model_name,
            jobName=f"{model_name}-import-{int(time.time())}",
            modelDataSource={
                "s3DataSource": {
                    "s3Uri": s3_uri
                }
            },
            roleArn=role_arn
        )
        
        print(f"Model import job created successfully. Job ARN: {response['jobArn']}")
        return response['jobArn']
        
    except Exception as e:
        print(f"Error creating model import job: {str(e)}")
        raise

def check_import_status(job_arn):
    try:
        response = bedrock.get_model_import_job(
            jobIdentifier=job_arn
        )
        # Print full response for debugging
        print("Full response:")
        print(json.dumps(response, indent=2, default=str))
        
        # Try to get status from response
        if 'status' in response:
            return response['status']
        elif 'modelImportJob' in response and 'status' in response['modelImportJob']:
            return response['modelImportJob']['status']
        else:
            print("Status not found in response structure")
            return None
            
    except Exception as e:
        print(f"Error checking job status: {str(e)}")
        raise

# Start the import
job_arn = start_model_import()

# Monitor the import status
while True:
    try:
        status = check_import_status(job_arn)
        if status:
            print(f"Import status: {status}")
            if status in ['Completed', 'Failed']:
                break
        else:
            print("Could not determine status")
            break
        time.sleep(60)  # Check every minute
    except Exception as e:
        print(f"Error in monitoring loop: {str(e)}")
        break

## Test Inference with Imported Model

In [ ]:
# Initialize Bedrock Runtime client
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-west-2'
)

# Test prompt
prompt = "What does Mistral AI do?"

# Prepare the request body
request_body = {
    "prompt": prompt,
    "max_tokens": 500,
    "temperature": 0.2,
    "top_k": 50
}

try:
    # Invoke the model
    response = bedrock_runtime.invoke_model(
        modelId='YOUR_MODEL_ARN', #from the bedrock console under imported models
        body=json.dumps(request_body)
    )
    
    # Parse and print the response
    response_body = json.loads(response['body'].read())
    print("Response:")
    print(response_body)
    text_content = response_body['outputs'][0]['text']
    print("Response text:\n", text_content)
    
    
except Exception as e:
    print(f"Error: {str(e)}")

## Cleanup

The following resources were created and may need cleanup:

- Local temporary files in `./temp_model_files`
- S3 objects (model files uploaded to your bucket)
- IAM role "MyImportModelRole"
- IAM policy "S3BucketPolicy"
- Imported Bedrock model (optional cleanup)


In [ ]:
### Cleanup Resources

import boto3
import shutil
import os

# Initialize clients
bedrock = boto3.client('bedrock')
s3 = boto3.client('s3')
iam = boto3.client('iam')

# Set your variables (same as used earlier)
bucket_name = "YOUR_BUCKET_NAME"
prefix = "PREFIX"
model_name = "mistral-nemo-test"  # The name you used for the imported model
policy_name = "S3BucketPolicy"
role_name = "MyImportModelRole"
policy_arn = "YOUR_POLICY_ARN"  # The ARN of the policy you created

# 1. Delete local temporary files
print("Cleaning up local temporary files...")
if os.path.exists("./temp_model_files"):
    shutil.rmtree("./temp_model_files")
    print("✓ Local temporary files deleted")
else:
    print("No local temporary files found")

# 2. Delete S3 objects
print(f"\nCleaning up S3 objects in s3://{bucket_name}/{prefix}/...")
try:
    # List all objects with the prefix
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    if 'Contents' in response:
        # Create a list of objects to delete
        objects_to_delete = [{'Key': obj['Key']} for obj in response['Contents']]
        
        # Delete the objects
        s3.delete_objects(
            Bucket=bucket_name,
            Delete={'Objects': objects_to_delete}
        )
        print(f"✓ Deleted {len(objects_to_delete)} objects from S3")
    else:
        print("No objects found in the specified S3 prefix")
        
except Exception as e:
    print(f"Error deleting S3 objects: {str(e)}")

# 3. Optional: Delete the imported model from Bedrock
# CAUTION: Only do this if you no longer need the imported model
delete_model = input("Do you want to delete the imported Bedrock model? (yes/no): ")
if delete_model.lower() == 'yes':
    try:
        print(f"\nDeleting imported model {model_name}...")
        bedrock.delete_imported_model(
            modelIdentifier=model_name  # You might need to use the full ARN instead
        )
        print("✓ Imported model deleted")
    except Exception as e:
        print(f"Error deleting imported model: {str(e)}")
        print("You may need to delete the model manually from the Bedrock console")
else:
    print("\nSkipping model deletion")

# 4. Detach and delete IAM policy
print("\nCleaning up IAM resources...")
try:
    # Detach policy from role
    iam.detach_role_policy(
        RoleName=role_name,
        PolicyArn=policy_arn
    )
    print(f"✓ Policy {policy_name} detached from role {role_name}")
    
    # Delete policy
    iam.delete_policy(
        PolicyArn=policy_arn
    )
    print(f"✓ Policy {policy_name} deleted")
    
except Exception as e:
    print(f"Error cleaning up IAM policy: {str(e)}")

# 5. Delete IAM role
try:
    iam.delete_role(
        RoleName=role_name
    )
    print(f"✓ Role {role_name} deleted")
except Exception as e:
    print(f"Error deleting IAM role: {str(e)}")
    print("Note: If the role has any remaining attached policies, it cannot be deleted")

print("\nCleanup complete!")
